In [10]:
pip install fosforml

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install pandas
!pip install snowflake-ml-python
!pip install requests

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
pip install ydata-profiling --upgrade

     |████████████████████████████████| 358kB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 14.0MB/s eta 0:00:01
     |████████████████████████████████| 696kB 53.4MB/s eta 0:00:01
     |████████████████████████████████| 440kB 87.3MB/s eta 0:00:01
     |████████████████████████████████| 296kB 85.4MB/s eta 0:00:01
     |████████████████████████████████| 112kB 99.0MB/s eta 0:00:01
     |████████████████████████████████| 296kB 87.3MB/s eta 0:00:01
     |████████████████████████████████| 10.8MB 92.6MB/s eta 0:00:01
     |████████████████████████████████| 522kB 80.2MB/s eta 0:00:01
     |████████████████████████████████| 43.9MB 66.1MB/s eta 0:00:010:00:01    |███████████▉                    | 16.1MB 66.1MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 77.8MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 38.4MB/s eta 0:00:01
     |████████████████████████████████| 4.5MB 94.4MB/s eta 0:00:01
     |████████████████████████████████| 235kB 93.0MB/s 

In [13]:
import pandas as pd
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [14]:
import seaborn as sns

In [15]:
my_session.connection.database

'ASSORTMENT_PLANNING'

In [16]:
my_session.connection.schema

'CPG_BRONZE'

In [17]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [18]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [20]:
type(sf_df)

snowflake.snowpark.dataframe.DataFrame

In [21]:
df=sf_df.to_pandas()

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966400 entries, 0 to 966399
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   MNTH_CODE         966400 non-null  int32  
 1   TRANS_DATE        966400 non-null  object 
 2   START_DATE        966400 non-null  object 
 3   SALES_VALUE       966400 non-null  float64
 4   SALES_UNITS       966400 non-null  int16  
 5   SALES_VOLUME      966400 non-null  float64
 6   SALES_PTR_VALUE   966400 non-null  float64
 7   OC_CODE           966400 non-null  int32  
 8   DISTRIBUTOR_CODE  966400 non-null  object 
 9   OUTLET_CODE       966400 non-null  object 
 10  CITY              966400 non-null  object 
 11  STATE             966400 non-null  object 
 12  COUNTY            966400 non-null  object 
 13  STREET            966400 non-null  object 
 14  PRODUCT_CODE      966400 non-null  object 
 15  CATEGORY          966400 non-null  object 
 16  SUBCATEGORY       96

In [ ]:
missing_percentage = df.isnull().sum() / df.shape[0] * 100
missing_percentage